In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import Tensor

import numpy as np
import tensorflow as tf
from scipy.misc import imread, imresize, imsave, imshow
from imagenet_classes import class_names 
import matplotlib.pyplot as plt
import json
import pickle
import os
import re



%matplotlib inline

# Import

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
#  load fine grained classifcation dataset from memory

In [9]:
meta_1 = json.load(open("/home/tuna/Projects/CS601R/Project3/lab10/lib/cub_metadata.json", "r"))
meta_2 = pickle.load(open("/home/tuna/Projects/CS601R/Project3/lab10/lib/cub_metadata.pkl", "r"))

In [10]:
#  remove number prefixes from names
for inst in meta_2['class_names']:
    meta_2['class_names'][inst] = meta_2['class_names'][inst][4:].lower()
    

In [11]:
meta_2['class_names'][1]

'black_footed_albatross'

In [12]:
#  clean up imageNet classes for comparison
standardized_class_names = []
for name in class_names:
    standardized_class_names.append("_".join(re.split("[^a-zA-Z\d:]+", name.lower())))
    


In [13]:
rootdir = '/home/tuna/Projects/CS601R/Project3/lab10/lib/images_sq224'

all_ims_original = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        all_ims_original.append( imresize(imread(os.path.join(subdir, file), mode='RGB'), (224, 224)))
    
all_ims_original = np.asarray(all_ims_original)

In [14]:
#  create training/test sets

test_data_original = []
test_classes_original = []

# for i in range(1, len(meta_2['train_test_split']['testing'])):
for i in range(1, 5):
    test_data_original.extend(all_ims_original[meta_2['train_test_split']['testing'][i]])
    test_classes_original.extend([i] * len(meta_2['train_test_split']['testing'][i])) 
    
    
    
train_data_original = []
train_classes_original = []

# for i in range(1, len(meta_2['train_test_split']['training'])):
for i in range(1, 5):
    train_data_original.extend(all_ims_original[meta_2['train_test_split']['training'][i]])
    train_classes_original.extend([i] * len(meta_2['train_test_split']['training'][i]))   

In [15]:
#  get class names
def get_class_names(passed_indexes):
    class_names = []
    for i in passed_indexes:
        class_names.append(meta_2['class_names'][i])
    
    return class_names

test_classes_original_names = get_class_names(test_classes_original)
train_classes_original_names = get_class_names(train_classes_original)

In [16]:
#  one hot encoded versions

train_classes_ohe = np.zeros((len(train_classes_original),5))
for i,c in enumerate(train_classes_original):
    train_classes_ohe[i,c] = 1.0
    
test_classes_ohe = np.zeros((len(test_classes_original),5))
for i,c in enumerate(test_classes_original):
    test_classes_ohe[i,c] = 1.0
    
    
train_classes_ohe[0]

array([ 0.,  1.,  0.,  0.,  0.])

In [17]:
    
"""Define get next batch method"""
def get_next_batch(data, classes, num_to_select):
  #  shuffle with labels
  data_shuffled_mat =  np.asarray(zip(data, classes))
  np.random.shuffle(data_shuffled_mat)
  
  data_mat_selected = np.asarray(list(data_shuffled_mat[:num_to_select,0]))
  data_classes_selected = np.asarray(list(data_shuffled_mat[:num_to_select,1]))
  
  return data_mat_selected.astype(float), data_classes_selected.astype(float)



In [18]:
tmp_test_batch_data,tmp_test_batch_labels  = get_next_batch(train_data_original, train_classes_ohe, 2)
tmp_test_batch_labels

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

# Model

# BCNN with BIRDS

In [21]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_4x4(x):
  return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    
    
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
















with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 5])
    x_image = tf.reshape(x, [-1,224, 224, 3])

    
    with tf.name_scope('bilinear_model_1') as scope:
        W_conv1_bnn1 = weight_variable([5, 5, 3, 32])
        b_conv1_bnn1 = bias_variable([32])
        h_conv1_bnn1 = tf.nn.relu(conv2d(x_image, W_conv1_bnn1) + b_conv1_bnn1)
        h_pool1_bnn1 = max_pool_2x2(h_conv1_bnn1)




        W_conv2_bnn1 = weight_variable([5, 5, 32, 40])
        b_conv2_bnn1 = bias_variable([40])
        h_conv2_bnn1 = tf.nn.relu(conv2d(h_pool1_bnn1, W_conv2_bnn1) + b_conv2_bnn1)
        h_pool2_bnn1 = max_pool_2x2(h_conv2_bnn1)




        W_conv3_bnn1 = weight_variable([5, 5, 40, 16])
        b_conv3_bnn1 = bias_variable([16])
        h_conv3_bnn1 = tf.nn.relu(conv2d(h_pool2_bnn1, W_conv3_bnn1) + b_conv3_bnn1)
    #     h_pool3 = max_pool_2x2(h_conv3)





        W_fc1_bnn1 = weight_variable([56 * 56 * 16, 20])
        b_fc1_bnn1 = bias_variable([20])
        h_conv3_flat_bnn1 = tf.reshape(h_conv3_bnn1, [-1, 56 * 56 * 16])
        h_fc1_bnn1 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn1, W_fc1_bnn1) + b_fc1_bnn1)



        keep_prob = tf.placeholder(tf.float32)
        y_raw_bnn1 = h_fc1_bnn1

        
        
        
        
        
    with tf.name_scope('bilinear_model_2') as scope:
        W_conv1_bnn2 = weight_variable([5, 5, 3, 32])
        b_conv1_bnn2 = bias_variable([32])
        h_conv1_bnn2 = tf.nn.relu(conv2d(x_image, W_conv1_bnn2) + b_conv1_bnn2)
        h_pool1_bnn2 = max_pool_2x2(h_conv1_bnn2)




        W_conv2_bnn2 = weight_variable([5, 5, 32, 42])
        b_conv2_bnn2 = bias_variable([42])
        h_conv2_bnn2 = tf.nn.relu(conv2d(h_pool1_bnn2, W_conv2_bnn2) + b_conv2_bnn2)
        h_pool2_bnn2 = max_pool_2x2(h_conv2_bnn2)




        W_conv3_bnn2 = weight_variable([5, 5, 42, 16])
        b_conv3_bnn2 = bias_variable([16])
        h_conv3_bnn2 = tf.nn.relu(conv2d(h_pool2_bnn2, W_conv3_bnn2) + b_conv3_bnn2)
    #     h_pool3 = max_pool_2x2(h_conv3)





        W_fc1_bnn2 = weight_variable([56 * 56 * 16, 20])
        b_fc1_bnn2 = bias_variable([20])
        h_conv3_flat_bnn2 = tf.reshape(h_conv3_bnn2, [-1, 56 * 56 * 16])
        h_fc1_bnn2 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn2, W_fc1_bnn2) + b_fc1_bnn2)



        keep_prob = tf.placeholder(tf.float32)
        y_raw_bnn2 = h_fc1_bnn2



    with tf.name_scope('outer_product') as scope:


        def batch_outer_product(x, y):
            print("it begins")
            print(x.shape)
            print(y.shape)

    #         x_transposed = tf.transpose(x)
            x_transposed = x
    #         print x_transposed.get_shape()  # ==> [N, U]

            x_transposed_as_matrix_batch = tf.expand_dims(x_transposed, 2)
            print(x_transposed_as_matrix_batch.shape)
    #         print x_transposed_as_matrix_batch.get_shape()  # ==> [N, U, 1]

            y_as_matrix_batch = tf.expand_dims(y, 1)
            print(y_as_matrix_batch.shape)

    #         print y_as_matrix_batch.get_shape()  # ==> [N, 1, V]

            result = tf.matmul(x_transposed_as_matrix_batch, y_as_matrix_batch)

            print(result.shape)
    #         result = tf.batch_matmul(x_transposed_as_matrix_batch, y_as_matrix_batch)
        #         print result.get_shape()  # ==> [N, U, V]
            print("It ends")
            return result



        #  shape 50 x 10, to 50 x 100
        print(y_raw_bnn1.shape)
        y_outer_product = batch_outer_product(y_raw_bnn1, y_raw_bnn2)
        print(y_outer_product.shape)


    #     #  shape = 50 x 100
        y_outer_product_vec = tf.nn.relu(tf.sqrt(tf.reshape(y_outer_product, [x_batch_size,-1])))
        print(y_outer_product_vec.shape)

    
    
    
    #  final fc layer transformation to 10 classes
    W_fc3 = weight_variable([400, 5])
    b_fc3 = bias_variable([5])
    y_fc3 = tf.matmul(y_outer_product_vec, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



    
    
    
    
    
#  FINE AFTER HERE

    
    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1_bnn1) +\
                beta*l2_loss(b_conv1_bnn1) +\
                beta*l2_loss(W_conv2_bnn1) +\
                beta*l2_loss(b_conv2_bnn1) +\
                beta*l2_loss(W_conv3_bnn1) +\
                beta*l2_loss(b_conv3_bnn1) +\
                beta*l2_loss(W_fc1_bnn1) +\
                beta*l2_loss(b_fc1_bnn1) +\
                beta*l2_loss(W_conv1_bnn2) +\
                beta*l2_loss(b_conv1_bnn2) +\
                beta*l2_loss(W_conv2_bnn2) +\
                beta*l2_loss(b_conv2_bnn2) +\
                beta*l2_loss(W_conv3_bnn2) +\
                beta*l2_loss(b_conv3_bnn2) +\
                beta*l2_loss(W_fc1_bnn2) +\
                beta*l2_loss(b_fc1_bnn2) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)
#                 beta*l2_loss(W_fc2) +\
#                 beta*l2_loss(b_fc2) +\



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/full_bilinear_test_birds_1', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(100):
#   batch = mnist.train.next_batch(batch_size)
  batch_xs, batch_ys = get_next_batch(train_data_original, train_classes_ohe, batch_size)

    
    
#   print("Batch", i)
  if i%1 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _, outshape, wc111, bc111, yc, hp1 = sess.run([merged_summary_op,train_step,out_shape, W_conv1_bnn1, b_conv1_bnn1,  y_conv, h_pool1_bnn1], feed_dict={x:batch_xs, y_: batch_ys, keep_prob: 0.5})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("yc", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x:test_data_original, y_: test_classes_ohe, keep_prob: 1.0}))





summary_writer.close()
sess.close()

(?, 20)
it begins
(?, 20)
(?, 20)
(?, 20, 1)
(?, 1, 20)
(?, 20, 20)
It ends
(?, 20, 20)
(?, ?)
(?, 5)
step 0, training accuracy 0.24
step 1, training accuracy 0.24
step 2, training accuracy 0.18
step 3, training accuracy 0.26
step 4, training accuracy 0.24
step 5, training accuracy 0.22
step 6, training accuracy 0.32
step 7, training accuracy 0.34
step 8, training accuracy 0.18
step 9, training accuracy 0.22
step 10, training accuracy 0.26
step 11, training accuracy 0.24
step 12, training accuracy 0.3
step 13, training accuracy 0.26
step 14, training accuracy 0.3
step 15, training accuracy 0.2
step 16, training accuracy 0.26
step 17, training accuracy 0.26
step 18, training accuracy 0.32
step 19, training accuracy 0.28
step 20, training accuracy 0.24
step 21, training accuracy 0.24
step 22, training accuracy 0.34
step 23, training accuracy 0.24
step 24, training accuracy 0.22
step 25, training accuracy 0.24
step 26, training accuracy 0.32
step 27, training accuracy 0.14
step 28, traini

In [3]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_4x4(x):
  return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    
    
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
















with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 5])
    x_image = tf.reshape(x, [-1,224, 224, 3])

    
    with tf.name_scope('bilinear_model_1') as scope:
        W_conv1_bnn1 = weight_variable([5, 5, 1, 32])
        b_conv1_bnn1 = bias_variable([32])
        h_conv1_bnn1 = tf.nn.relu(conv2d(x_image, W_conv1_bnn1) + b_conv1_bnn1)
        h_pool1_bnn1 = max_pool_2x2(h_conv1_bnn1)




        W_conv2_bnn1 = weight_variable([5, 5, 32, 40])
        b_conv2_bnn1 = bias_variable([40])
        h_conv2_bnn1 = tf.nn.relu(conv2d(h_pool1_bnn1, W_conv2_bnn1) + b_conv2_bnn1)
        h_pool2_bnn1 = max_pool_2x2(h_conv2_bnn1)




        W_conv3_bnn1 = weight_variable([5, 5, 40, 16])
        b_conv3_bnn1 = bias_variable([16])
        h_conv3_bnn1 = tf.nn.relu(conv2d(h_pool2_bnn1, W_conv3_bnn1) + b_conv3_bnn1)
    #     h_pool3 = max_pool_2x2(h_conv3)





        W_fc1_bnn1 = weight_variable([56 * 56 * 16, 20])
        b_fc1_bnn1 = bias_variable([20])
        h_conv3_flat_bnn1 = tf.reshape(h_conv3_bnn1, [-1, 56 * 56 * 16])
        h_fc1_bnn1 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn1, W_fc1_bnn1) + b_fc1_bnn1)



        keep_prob = tf.placeholder(tf.float32)
        y_raw_bnn1 = h_fc1_bnn1

        
        
        
        
        
    with tf.name_scope('bilinear_model_2') as scope:
        W_conv1_bnn2 = weight_variable([5, 5, 1, 32])
        b_conv1_bnn2 = bias_variable([32])
        h_conv1_bnn2 = tf.nn.relu(conv2d(x_image, W_conv1_bnn2) + b_conv1_bnn2)
        h_pool1_bnn2 = max_pool_2x2(h_conv1_bnn2)




        W_conv2_bnn2 = weight_variable([5, 5, 32, 42])
        b_conv2_bnn2 = bias_variable([42])
        h_conv2_bnn2 = tf.nn.relu(conv2d(h_pool1_bnn2, W_conv2_bnn2) + b_conv2_bnn2)
        h_pool2_bnn2 = max_pool_2x2(h_conv2_bnn2)




        W_conv3_bnn2 = weight_variable([5, 5, 42, 16])
        b_conv3_bnn2 = bias_variable([16])
        h_conv3_bnn2 = tf.nn.relu(conv2d(h_pool2_bnn2, W_conv3_bnn2) + b_conv3_bnn2)
    #     h_pool3 = max_pool_2x2(h_conv3)





        W_fc1_bnn2 = weight_variable([56 * 56 * 16, 20])
        b_fc1_bnn2 = bias_variable([20])
        h_conv3_flat_bnn2 = tf.reshape(h_conv3_bnn2, [-1, 56 * 56 * 16])
        h_fc1_bnn2 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn2, W_fc1_bnn2) + b_fc1_bnn2)



        keep_prob = tf.placeholder(tf.float32)
        y_raw_bnn2 = h_fc1_bnn2



    with tf.name_scope('outer_product') as scope:


        def batch_outer_product(x, y):
            print("it begins")
            print(x.shape)
            print(y.shape)

    #         x_transposed = tf.transpose(x)
            x_transposed = x
    #         print x_transposed.get_shape()  # ==> [N, U]

            x_transposed_as_matrix_batch = tf.expand_dims(x_transposed, 2)
            print(x_transposed_as_matrix_batch.shape)
    #         print x_transposed_as_matrix_batch.get_shape()  # ==> [N, U, 1]

            y_as_matrix_batch = tf.expand_dims(y, 1)
            print(y_as_matrix_batch.shape)

    #         print y_as_matrix_batch.get_shape()  # ==> [N, 1, V]

            result = tf.matmul(x_transposed_as_matrix_batch, y_as_matrix_batch)

            print(result.shape)
    #         result = tf.batch_matmul(x_transposed_as_matrix_batch, y_as_matrix_batch)
        #         print result.get_shape()  # ==> [N, U, V]
            print("It ends")
            return result



        #  shape 50 x 10, to 50 x 100
        print(y_raw_bnn1.shape)
        y_outer_product = batch_outer_product(y_raw_bnn1, y_raw_bnn2)
        print(y_outer_product.shape)


    #     #  shape = 50 x 100
        y_outer_product_vec = tf.nn.relu(tf.sqrt(tf.reshape(y_outer_product, [x_batch_size,-1])))
        print(y_outer_product_vec.shape)

    
    
    
    #  final fc layer transformation to 10 classes
    W_fc3 = weight_variable([400, 10])
    b_fc3 = bias_variable([10])
    y_fc3 = tf.matmul(y_outer_product_vec, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



    
    
    
    
    
#  FINE AFTER HERE

    
    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1_bnn1) +\
                beta*l2_loss(b_conv1_bnn1) +\
                beta*l2_loss(W_conv2_bnn1) +\
                beta*l2_loss(b_conv2_bnn1) +\
                beta*l2_loss(W_conv3_bnn1) +\
                beta*l2_loss(b_conv3_bnn1) +\
                beta*l2_loss(W_fc1_bnn1) +\
                beta*l2_loss(b_fc1_bnn1) +\
                beta*l2_loss(W_conv1_bnn2) +\
                beta*l2_loss(b_conv1_bnn2) +\
                beta*l2_loss(W_conv2_bnn2) +\
                beta*l2_loss(b_conv2_bnn2) +\
                beta*l2_loss(W_conv3_bnn2) +\
                beta*l2_loss(b_conv3_bnn2) +\
                beta*l2_loss(W_fc1_bnn2) +\
                beta*l2_loss(b_fc1_bnn2) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)
#                 beta*l2_loss(W_fc2) +\
#                 beta*l2_loss(b_fc2) +\



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/full_bilinear_test_birds_1', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(100):
#   batch = mnist.train.next_batch(batch_size)
  batch_xs, batch_ys = get_next_batch(train_data_original, train_classes_ohe, batch_size)

    
    
#   print("Batch", i)
  if i%1 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _, outshape, wc111, bc111, yc, hp1 = sess.run([merged_summary_op,train_step,out_shape, W_conv1_bnn1, b_conv1_bnn1,  y_conv, h_pool1_bnn1], feed_dict={x:batch_xs, y_: batch_ys, keep_prob: 0.5})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("yc", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x:test_data_original, y_: test_classes_ohe, keep_prob: 1.0}))





summary_writer.close()
sess.close()

(?, 20)
it begins
(?, 20)
(?, 20)
(?, 20, 1)
(?, 1, 20)
(?, 20, 20)
It ends
(?, 20, 20)
(?, ?)
(?, 10)
step 0, training accuracy 0.12
step 100, training accuracy 0.12
step 200, training accuracy 0.16
step 300, training accuracy 0.14
step 400, training accuracy 0.12
step 500, training accuracy 0.1
step 600, training accuracy 0.12
step 700, training accuracy 0.12
step 800, training accuracy 0.12
step 900, training accuracy 0.06
step 1000, training accuracy 0.12
step 1100, training accuracy 0.12
step 1200, training accuracy 0.14
step 1300, training accuracy 0.16
step 1400, training accuracy 0.14
step 1500, training accuracy 0.1
step 1600, training accuracy 0.08
step 1700, training accuracy 0.16
step 1800, training accuracy 0.08
step 1900, training accuracy 0.12
step 2000, training accuracy 0.06
step 2100, training accuracy 0.12
step 2200, training accuracy 0.06
step 2300, training accuracy 0.12
step 2400, training accuracy 0.04
step 2500, training accuracy 0.14
step 2600, training accurac

KeyboardInterrupt: 

In [3]:



sess = tf.InteractiveSession()    
   
batch_size = 50
    

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
#     return max_pool_custom_crazy_slow(x)


# def max_pool_custom_crazy_slow(x, batch_size= 50):
#   print(x.shape)
#   N, H, W, C = x.shape
#   N = batch_size
#   pool_height, pool_width, stride = 2, 2, 2
#   assert (H - pool_height) % stride == 0
#   assert (W - pool_width) % stride == 0


#   out_height = (H - pool_height) / stride + 1
#   out_width = (W - pool_width) / stride + 1
  
  
    
#   output = []

# #   out = tf.zeros(tf.shape(x))

# #   out = np.zeros((N, out_height, out_width, C))
#   #  for each channel
#   print("1")
#   for c in xrange(C):
#       print(c)
#       #  for each passed image
#       for n in xrange(N):
#           idx_i = 0
#           for i in xrange(pool_height, H+1, stride):
#               idx_j = 0
#               for j in xrange(pool_width, W+1, stride):
#                   field = x[n,i-pool_height:i,j-pool_width:j,c]
#                   output.append(tf.reduce_max(field))
# #                   out[n,idx_i, idx_j, c] = tf.reduce_max(field)
#                   idx_j += 1
#               idx_i += 1
            
#   output = tf.reshape(tf.stack(output), [N, H, W, C])  ## second is shape
            
#   return output

    

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
#     return conv_forward_naive(x, W)


# def conv2d(x, W, HH,WW,C,F):
# #   return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
#     return conv_forward_naive(x, W, HH,WW,C,F)

# def conv_forward_naive(x, w, HH,WW,C,F,  batch_size=50):
#   """
#   A naive implementation of the forward pass for a convolutional layer.
#   The input consists of N data points, each with C channels, height H and width
#   W. We convolve each input with F different filters, where each filter spans
#   all C channels and has height HH and width WW.
#   Input:
#   - x: Input data of shape (N, C, H, W)
#   - w: Filter weights of shape (F, C, HH, WW)
#   - b: Biases, of shape (F,)
#   - conv_param: A dictionary with the following keys:
#     - 'stride': The number of pixels between adjacent receptive fields in the
#       horizontal and vertical directions.
#     - 'pad': The number of pixels that will be used to zero-pad the input.
#   Returns a tuple of:
#   - out: Output data, of shape (N, F, H', W') where H' and W' are given by
#     H' = 1 + (H + 2 * pad - HH) / stride
#     W' = 1 + (W + 2 * pad - WW) / stride
#   - cache: (x, w, b, conv_param)
#   """

#   N, H, W, C = x.get_shape().as_list() 
# #   HH,WW,C,F = w.shape
#   stride, pad = 1, 1
#   print(x.shape)
#   N = batch_size
#   H_prime = 1. + float(H + 2 * pad - HH) / float(stride)
#   W_prime = 1. + float(W + 2 * pad - WW) / float(stride)
#   assert H_prime % 1 == 0
#   assert W_prime % 1 == 0
#   H_prime,W_prime = int(H_prime), int(W_prime)


# #   # pad input array
# #   x_padded = np.pad(x, ((0,0), (0,0), (pad, pad), (pad, pad)), 'constant')
# #   H_padded, W_padded = x_padded.shape[2], x_padded.shape[3]

#   #  no padding
#   x_padded = x
#   H_padded = H
#   W_padded = W


#   # naive implementation of im2col
# #   x_cols = None
#   print("bob")
#   x_cols = []
#   for i in xrange(HH, H_padded+1, stride):
#     print(i)
#     for j in xrange(WW, W_padded+1, stride):
#       for n in xrange(N):
#         field = tf.reshape(x_padded[n,i-HH:i, j-WW:j,:], (1,C*HH*WW))    
#         if x_cols is None:
# #             field = tf.expand_dims(field, 0)
# #             x_cols = field
#           x_cols.append(field)
#         else:
# #             field = tf.expand_dims(field, 0)
# #             x_cols = tf.stack((x_cols, field))
#           x_cols.append(field)

    
#   #  every n (at HH, WW, w/ c), then every WW, then every HH
    
    
#   x_cols = tf.stack(x_cols)
  
#   # x_cols shape: (HH * WW * C) x (H_prime * W_prime * N)
#   x_cols = tf.transpose(x_cols)

#   #w2col, get w into shape of (F) x (HH * WW * C) 
#   w_cols = tf.reshape(w, (F, C*HH *WW))
  
  
#   # out_cols shape = (F) x (H_prime * W_prime * N)
#   out_cols = np.dot(w_cols, x_cols)

#   # out shape = N x F x H' x W'
#   out = tf.reshape(out_cols, (F, H_prime, W_prime, N))
# #   out = out.transpose(3, 0, 1, 2) # (N, F, H', W')
#   out = out.transpose(3, 1, 2, 0) # (N, F, H', W')


#   return out
   
    
    
    
    
    
    
    
def weight_variable(shape, name=None):
  if name is not None:
    initial = tf.truncated_normal(shape, stddev=1e-4)
    return tf.Variable(initial, name=name)
  else:
    initial = tf.truncated_normal(shape, stddev=1e-4)
    return tf.Variable(initial)

def bias_variable(shape, name=None):
  if name is not None:
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)
  else:
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
        















In [4]:


with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 784])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 10])


    W_conv1 = weight_variable([5, 5, 1, 32], name='W_conv1')
    b_conv1 = bias_variable([32], name='b_conv1')

    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#     h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 5, 5, 1, 32) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)




    W_conv2 = weight_variable([5, 5, 32, 32], name='W_conv2')
    b_conv2 = bias_variable([32], name='b_conv2')
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 5, 5, 32, 64) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)





    W_fc1 = weight_variable([7 * 7 * 32, 20], name='W_fc1')
    b_fc1 = bias_variable([20], name='b_fc1')

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)




    keep_prob = tf.placeholder(tf.float32)

    y_raw = h_fc1
    
    
    
    
    
    
    # DONT SAVE THESE WEIGHTS
    
    #  final fc layer transformation to 10 classes
#     W_fc3 = weight_variable([100, 10])
    W_fc3 = weight_variable([20, 10], name='W_fc3')
    b_fc3 = bias_variable([10], name='b_fc3')
    y_fc3 = tf.matmul(y_raw, W_fc3) + b_fc3
#     y_fc3 = tf.matmul(h_fc1, W_fc3) + b_fc3

    







    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)


    
    
#  set up saver
saver = tf.train.Saver({"Wx_B/W_conv1": W_conv1, "Wx_B/b_conv1": b_conv1, "Wx_B/W_conv2": W_conv2, "Wx_B/b_conv2": b_conv2,
                       "Wx_B/W_fc1": W_fc1, "Wx_B/b_fc1": b_fc1,})
# saver = tf.train.Saver()


saver.restore(sess, "/tmp/model2.ckpt")



    
    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1) +\
                beta*l2_loss(W_conv2) +\
                beta*l2_loss(b_conv1) +\
                beta*l2_loss(b_conv2) +\
                beta*l2_loss(W_fc1) +\
                beta*l2_loss(b_fc1) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/4_layers_test_14_pre_train_save_', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(5000):
  batch = mnist.train.next_batch(batch_size)
#   print("Batch", i)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _, outshape, wc111, bc111, yc, hp1 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1,  y_conv, h_pool1], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("y_softmax", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))



#  save the weights
# saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
  # Save the variables to disk.
# save_path = saver.save(sess, "/tmp/model2.ckpt")
# print("Model saved in file: %s" % save_path)








summary_writer.close()
sess.close()

(?, 10)
step 0, training accuracy 0.04
step 100, training accuracy 0.12
step 200, training accuracy 0.16
step 300, training accuracy 0.14
step 400, training accuracy 0.12
step 500, training accuracy 0.18
step 600, training accuracy 0.16
step 700, training accuracy 0.24
step 800, training accuracy 0.22
step 900, training accuracy 0.16
step 1000, training accuracy 0.32
step 1100, training accuracy 0.2
step 1200, training accuracy 0.34
step 1300, training accuracy 0.4
step 1400, training accuracy 0.52
step 1500, training accuracy 0.56
step 1600, training accuracy 0.68
step 1700, training accuracy 0.54
step 1800, training accuracy 0.66
step 1900, training accuracy 0.72
step 2000, training accuracy 0.84
step 2100, training accuracy 0.74
step 2200, training accuracy 0.78
step 2300, training accuracy 0.72
step 2400, training accuracy 0.76
step 2500, training accuracy 0.8
step 2600, training accuracy 0.78
step 2700, training accuracy 0.74
step 2800, training accuracy 0.9
step 2900, training ac

In [5]:


with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 784])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 10])


    W_conv1 = weight_variable([5, 5, 1, 32], name='W_conv1')
    b_conv1 = bias_variable([32], name='b_conv1')

    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#     h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 5, 5, 1, 32) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)




    W_conv2 = weight_variable([5, 5, 32, 32], name='W_conv2')
    b_conv2 = bias_variable([32], name='b_conv2')
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 5, 5, 32, 64) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)





    W_fc1 = weight_variable([7 * 7 * 32, 20], name='W_fc1')
    b_fc1 = bias_variable([20], name='b_fc1')

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)




    keep_prob = tf.placeholder(tf.float32)

    y_raw = h_fc1
    
    
    
    
#  set up saver
saver = tf.train.Saver({"Wx_B/W_conv1": W_conv1, "Wx_B/b_conv1": b_conv1, "Wx_B/W_conv2": W_conv2, "Wx_B/b_conv2": b_conv2,
                       "Wx_B/W_fc1": W_fc1, "Wx_B/b_fc1": b_fc1,})
# saver = tf.train.Saver()


saver.restore(sess, "/tmp/model2.ckpt")



#  FINE BEFORE HERE



with tf.name_scope('Wx_B') as scope:


        
    def batch_outer_product(x, y):
#         print x.get_shape()  # ==> [U, N]
#         print y.get_shape()  # ==> [N, V]
        print("it begins")
        print(x.shape)
        print(y.shape)




#         x_transposed = tf.transpose(x)
        x_transposed = x
#         print x_transposed.get_shape()  # ==> [N, U]

        x_transposed_as_matrix_batch = tf.expand_dims(x_transposed, 2)
        print(x_transposed_as_matrix_batch.shape)
#         print x_transposed_as_matrix_batch.get_shape()  # ==> [N, U, 1]

        y_as_matrix_batch = tf.expand_dims(y, 1)
        print(y_as_matrix_batch.shape)

#         print y_as_matrix_batch.get_shape()  # ==> [N, 1, V]

        result = tf.matmul(x_transposed_as_matrix_batch, y_as_matrix_batch)
    
#         outer_products = []
#         N, H = tf.shape(x)[0], tf.shape(x)[1]
#         W = tf.shape(y)[1]
#         for i in tf.range(x_batch_size):
#             outer_products.append(tf.matmul(x_transposed_as_matrix_batch[i], y_as_matrix_batch[i]))
    
#         result = tf.reshape(tf.pack(outer_products), shape=(N, H, W))
    
        print(result.shape)
#         result = tf.batch_matmul(x_transposed_as_matrix_batch, y_as_matrix_batch)
    #         print result.get_shape()  # ==> [N, U, V]
        print("It ends")
        return result
    
    
    
    #  shape 50 x 10, to 50 x 100
    print(y_raw.shape)
    y_outer_product = batch_outer_product(y_raw, y_raw)
    print(y_outer_product.shape)

    
#     #  shape = 50 x 100
    y_outer_product_vec = tf.reshape(y_outer_product, [x_batch_size,-1])
    y_outer_product_norm = tf.nn.relu(tf.sign(y_outer_product_vec) * tf.sqrt(tf.abs(y_outer_product_vec)))
    print(y_outer_product_norm.shape)

    
    
    
    #  final fc layer transformation to 10 classes
#     W_fc3 = weight_variable([100, 10])
    W_fc3 = weight_variable([400, 10])
    b_fc3 = bias_variable([10])
    y_fc3 = tf.matmul(y_outer_product_norm, W_fc3) + b_fc3
#     y_fc3 = tf.matmul(h_fc1, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



    
    
    
    
    
#  FINE AFTER HERE

    
    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1) +\
                beta*l2_loss(W_conv2) +\
                beta*l2_loss(b_conv1) +\
                beta*l2_loss(b_conv2) +\
                beta*l2_loss(W_fc1) +\
                beta*l2_loss(b_fc1) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)
#                 beta*l2_loss(W_fc2) +\
#                 beta*l2_loss(b_fc2) +\



with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#  summary writers for debugging
summary_writer = tf.summary.FileWriter( '/home/tuna/Projects/CS678/Deep_Learning_Project_2/tf_logs/4_layers_test_15_outer_product_load_after_tested', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', cross_entropy )
merged_summary_op = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())






for i in range(150000000):
  batch = mnist.train.next_batch(batch_size)
#   print("Batch", i)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
#   summary, _, outshape, wc111, bc111, yop, yr, yopv, yc, hp1, hp2 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1, y_outer_product, y_raw, y_outer_product_vec, y_conv, h_pool1, h_pool2], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary, _, outshape, wc111, bc111, yc, hp1 = sess.run([merged_summary_op,train_step,out_shape, W_conv1, b_conv1,  y_conv, h_pool1], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  summary_writer.add_summary(summary, i)
#   print("out shape", outshape)
#   print("wc1", wc111)
#   print("bc1", bc111)
#   print("hp1", hp1)
#   print("hp2", hp2)
#   print("yr", yr)
#   print("yop", yop)
#   print("yopv", yopv)
#   print("y_softmax", yc)
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))



summary_writer.close()
sess.close()

(?, 20)
it begins
(?, 20)
(?, 20)
(?, 20, 1)
(?, 1, 20)
(?, 20, 20)
It ends
(?, 20, 20)
(?, ?)
(?, 10)
step 0, training accuracy 0.12
step 100, training accuracy 0.12
step 200, training accuracy 0.16
step 300, training accuracy 0.14
step 400, training accuracy 0.12
step 500, training accuracy 0.1
step 600, training accuracy 0.12
step 700, training accuracy 0.12
step 800, training accuracy 0.12
step 900, training accuracy 0.06
step 1000, training accuracy 0.12
step 1100, training accuracy 0.2
step 1200, training accuracy 0.1
step 1300, training accuracy 0.04
step 1400, training accuracy 0.06
step 1500, training accuracy 0.12
step 1600, training accuracy 0.06
step 1700, training accuracy 0.16
step 1800, training accuracy 0.22
step 1900, training accuracy 0.04
step 2000, training accuracy 0.12
step 2100, training accuracy 0.2
step 2200, training accuracy 0.08
step 2300, training accuracy 0.18
step 2400, training accuracy 0.1
step 2500, training accuracy 0.1
step 2600, training accuracy 0.

KeyboardInterrupt: 

(?, 20)
it begins
(?, 20)
(?, 20)
(?, 20, 1)
(?, 1, 20)
(?, 20, 20)
It ends
(?, 20, 20)
(?, ?)
(?, 10)
step 0, training accuracy 0.04
step 100, training accuracy 0.12
step 200, training accuracy 0.16
step 300, training accuracy 0.14
step 400, training accuracy 0.12
step 500, training accuracy 0.1
step 600, training accuracy 0.12
step 700, training accuracy 0.12
step 800, training accuracy 0.12
step 900, training accuracy 0.06
step 1000, training accuracy 0.12
step 1100, training accuracy 0.08
step 1200, training accuracy 0.08
step 1300, training accuracy 0.1
step 1400, training accuracy 0.08
step 1500, training accuracy 0.06
step 1600, training accuracy 0.12
step 1700, training accuracy 0.1
step 1800, training accuracy 0.14
step 1900, training accuracy 0.06
step 2000, training accuracy 0.1
step 2100, training accuracy 0.12
step 2200, training accuracy 0.12
step 2300, training accuracy 0.1
step 2400, training accuracy 0.1
step 2500, training accuracy 0.1
step 2600, training accuracy 0.1

KeyboardInterrupt: 

In [6]:
first = np.ones((3, 1, 2))
second = np.ones((3, 2, 1))
print(first)
print(second)

[[[ 1.  1.]]

 [[ 1.  1.]]

 [[ 1.  1.]]]
[[[ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]]]


In [7]:
np.matmul(first, second)

array([[[ 2.]],

       [[ 2.]],

       [[ 2.]]])

step 0, training accuracy 0.12
step 100, training accuracy 0.76
step 200, training accuracy 0.92
step 300, training accuracy 0.88
step 400, training accuracy 0.96
step 500, training accuracy 0.94
step 600, training accuracy 1
step 700, training accuracy 0.98
step 800, training accuracy 0.92
step 900, training accuracy 0.98
step 1000, training accuracy 0.94
step 1100, training accuracy 0.94
step 1200, training accuracy 0.98
step 1300, training accuracy 1
step 1400, training accuracy 0.98
test accuracy 0.9718


In [47]:
image = np.ones((2,10,10,3))
image[0,0,0,0] = 13
image[0,0,0,2] = 20
image[1,0,0,0] = 33
image[1,7,0,2] = 44
print(image.shape)
# image

(2, 10, 10, 3)


In [48]:
output = max_pool_forward_naive(image)
print(output.shape)
output

(2, 5, 5, 3)


array([[[[ 13.,   1.,  20.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.]],

        [[  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.]],

        [[  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.]],

        [[  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.]],

        [[  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.]]],


       [[[ 33.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.]],

        [[  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],
         [  1.,   1.,   1.],


In [ ]:
def max_pool_custom_crazy_slow(x, batch_size= 50):
  print(x.shape)
  N, H, W, C = x.shape
  N = batch_size
  pool_height, pool_width, stride = 2, 2, 2
  assert (H - pool_height) % stride == 0
  assert (W - pool_width) % stride == 0


  out_height = (H - pool_height) / stride + 1
  out_width = (W - pool_width) / stride + 1
  
  
    
  output = []

#   out = tf.zeros(tf.shape(x))

#   out = np.zeros((N, out_height, out_width, C))
  #  for each channel
  print("1")
  for c in xrange(C):
      print(c)
      #  for each passed image
      for n in xrange(N):
          print(n)
          idx_i = 0
            
          for i in xrange(pool_height, H+1, stride):
              print(i)
              idx_j = 0
              for j in xrange(pool_width, W+1, stride):
                  field = x[n,i-pool_height:i,j-pool_width:j,c]
                  output.append(tf.reduce_max(field))
#                   out[n,idx_i, idx_j, c] = tf.reduce_max(field)
                  idx_j += 1
              idx_i += 1
            
  output = tf.reshape(tf.pack(output), shape=(N, H, W, C))
            
  return output

    

In [ ]:

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

